In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

from api_keys import g_key

In [2]:
weather = "city.csv"
weather_df = pd.read_csv(weather, encoding="utf-8")
weather_df.head(50)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,dikson,73.5069,80.5464,12.31,88,100,17.72,RU,1648122229
1,cabo san lucas,22.8909,-109.9124,68.50,30,0,3.49,MX,1648122266
2,dunedin,-45.8742,170.5036,55.80,80,53,1.61,NZ,1648122557
3,rikitea,-23.1203,-134.9692,78.76,74,98,16.33,PF,1648122815
4,khatanga,71.9667,102.5000,0.77,96,100,6.40,RU,1648122815
5,punta arenas,-53.1500,-70.9167,46.51,76,0,18.41,CL,1648122815
6,talnakh,69.4865,88.3972,11.21,93,98,7.94,RU,1648122540
7,tiznit,29.5833,-9.5000,55.83,59,73,14.47,MA,1648122816
8,hithadhoo,-0.6000,73.0833,83.84,70,10,19.57,MV,1648122816
9,santa maria,-29.6842,-53.8069,69.10,94,75,4.61,BR,1648122817


In [3]:
gmaps.configure(api_key=g_key)

locations = weather_df[["Lat", "Lng"]].astype(float)
#weight = weather_df[["Humidity"]].astype(float)

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=weather_df["Humidity"], 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 3)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [4]:
weather_df_conditional = weather_df.loc[(weather_df["Max Temp"]<80) 
                                        & (weather_df["Max Temp"]>70) 
                                        & (weather_df["Wind Speed"]<10) 
                                        & (weather_df["Cloudiness"]==0)]
weather_df_conditional = weather_df_conditional.dropna()
weather_df_conditional

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
495,palmerston,-12.486,130.9833,76.98,94,0,5.75,AU,1648122956


In [5]:
hotel_df=weather_df_conditional
hotel_df["Hotel Name"] = " "
locations = hotel_df[["Lat", "Lng"]]
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
495,palmerston,-12.486,130.9833,76.98,94,0,5.75,AU,1648122956,


In [6]:
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    name_address = requests.get(base_url, params=params)
    
    name_address = name_address.json()
    
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")


In [7]:
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [8]:
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

fig

Figure(layout=FigureLayout(height='420px'))